In [1]:
# 1. 필요한 패키지 설치
!pip install fastapi pyngrok uvicorn transformers accelerate nest_asyncio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [2]:
# 2. 모델 로딩
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch

model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [3]:
# 모델과 토크나이저 로드
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# 3. 텍스트 생성 함수 (최적화 버전)
'''
def generate_text(prompt, max_length=512):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,               # 창의성 높임
        top_p=0.95,                   # nucleus sampling
        repetition_penalty=1.1,       # 반복 방지
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)
'''

'\ndef generate_text(prompt, max_length=512):\n    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)\n    output_ids = model.generate(\n        input_ids,\n        do_sample=True,\n        temperature=0.9,               # 창의성 높임\n        top_p=0.95,                   # nucleus sampling\n        repetition_penalty=1.1,       # 반복 방지\n        max_length=max_length,\n        pad_token_id=tokenizer.eos_token_id,\n    )\n    return tokenizer.decode(output_ids[0], skip_special_tokens=True)\n'

In [5]:
# 4. 정규표현식 파싱 함수 정의
import re

def parse_problems_from_text(text):
    pattern = r"Question\s*[:：]\s*(.*?)\s*Answer\s*[:：]\s*(.*?)(?=\nQuestion|\Z)"
    matches = re.findall(pattern, text, re.DOTALL)
    return [{"Question": q.strip(), "Answer": a.strip()} for q, a in matches]

In [6]:
# 5. FastAPI app 설정
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI()
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

class ProblemRequest(BaseModel):
    question_type: str
    topic: str
    count: int

class FeedbackRequest(BaseModel):
    question_id: str
    original_question: str
    updated_question: str
    feedback_type: str  # e.g. "incorrect_answer", "clarity", "typo"
    comment: str

@app.post("/generate")
def generate_problems(req: ProblemRequest):
    prompt = f"""
    Generate {req.count} {req.question_type} questions about {req.topic}.

    Question: ...
    Answer: ...
    (repeat {req.count} times)
    """

    result = generator(prompt, max_length=400, do_sample=True, temperature=0.8, top_p=0.9, repetition_penalty=1)
    output = result[0]["generated_text"]
    print("\nGPT Output:\n", output)

    try:
        problems = parse_problems_from_text(output)
        if not problems:
            raise ValueError("No questions could be parsed.")
    except Exception as e:
        problems = [{"Question": output if isinstance(output, str) else str(output), "Answer": f"⚠️ Parsing error: {str(e)}"}]

    return {
        "results": problems,
        "raw_output": output
    }

@app.post("/feedback")
def receive_feedback(feedback: FeedbackRequest):
    print("\n📥 Received Feedback:")
    print("Question ID:", feedback.question_id)
    print("Original:", feedback.original_question)
    print("Updated:", feedback.updated_question)
    print("Type:", feedback.feedback_type)
    print("Comment:", feedback.comment)

    # Here, you would typically store the feedback into a database or file for future model improvement.

    return {"status": "Feedback received", "question_id": feedback.question_id}

Device set to use cuda:0


In [7]:
# 6. Run ngrok + uvicorn server (Colab only)
from pyngrok import ngrok
import nest_asyncio
import uvicorn

ngrok.set_auth_token("2v1Fi5CEzLumREBpheNMIIepRlM_7uLFbq5PGe81hmEZiAe9K")  # Replace with your ngrok token
ngrok.kill()  # Kill previous tunnels

public_url = ngrok.connect(3000)
print("🔗 Public URL:", public_url.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=3000)


🔗 Public URL: https://028d-34-125-145-39.ngrok-free.app


INFO:     Started server process [2598]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)


INFO:     155.230.85.158:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "GET /openapi.json HTTP/1.1" 200 OK


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



GPT Output:
 
    Generate 2 Basic questions about Math.

    Question: ...
    Answer: ...
    (repeat 2 times)
        Question:...
        Answer:...
        (repeat 2 times)
            Question:...
            Answer:...
        (repeat 2 times)
            Question:...
            Answer:...
        (repeat 2 times)
            Question:...
            Answer:...
        (repeat 2 times)
            Question:...
            Answer:...
        (repeat 2 times)
            Question:...
            Answer:...
        (repeat 2 times)
            Question:...
            Answer:...
        (repeat 2 times)
            Question:...
            Answer:...
        (repeat 2 times)
            Question:...
            Answer:
INFO:     155.230.85.158:0 - "POST /generate HTTP/1.1" 200 OK

📥 Received Feedback:
Question ID: 12
Original: 1++1
Updated: 1+1
Type: typo
Comment: You have to type '+' 1 time.
INFO:     155.230.85.158:0 - "POST /feedback HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2598]
